In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

import spacy
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

# from transformers import pipeline

import warnings
warnings.filterwarnings('ignore')

In [54]:
df = pd.read_csv("https://media.githubusercontent.com/media/Tarshit/19CSE305-MachineLearning/main/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv", low_memory=False)
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,Amazon,B00ZV9PXP2,2017-09-03T00:00:00.000Z,NaN,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,Amazon,B00ZV9PXP2,2017-06-06T00:00:00.000Z,NaN,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,Amazon,B00ZV9PXP2,2018-04-20T00:00:00.000Z,NaN,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,Amazon,B00ZV9PXP2,2017-11-02T17:33:31.000Z,NaN,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,Amazon,B00ZV9PXP2,2018-04-24T00:00:00.000Z,NaN,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [3]:
df.shape

(5000, 24)

In [55]:
df2 = pd.read_csv('https://media.githubusercontent.com/media/Tarshit/19CSE305-MachineLearning/main/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv', low_memory=False)

In [5]:
df2.tail()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2016-12-07T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",NaN,True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,http://reviews.bestbuy.com/3545/5620410/review...
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-01-20T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",NaN,True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21,http://reviews.bestbuy.com/3545/5620410/review...
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-01-30T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",NaN,True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious,http://reviews.bestbuy.com/3545/5620410/review...
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-02-17T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",NaN,True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,http://reviews.bestbuy.com/3545/5620410/review...
28331,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2016-12-23T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",NaN,True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,"At ninety dollars, the expectionations are low...",You get what your paying for,Hung,http://reviews.bestbuy.com/3545/5620410/review...


In [6]:
df2.shape

(28332, 24)

In [7]:
data = df.append(df2)

In [8]:
data.tail()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,manufacturer,manufacturerNumber,reviews.date,reviews.dateAdded,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,reviews.didPurchase
28327,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2016-12-07T00:00:00.000Z,NaN,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,I got 2 of these for my 8 yr old twins. My 11 ...,Xmas gift,Mom2twinsplus1,http://reviews.bestbuy.com/3545/5620410/review...,NaN
28328,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-01-20T00:00:00.000Z,NaN,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,I bought this for my niece for a Christmas gif...,yes it is a great tablet.,fireman21,http://reviews.bestbuy.com/3545/5620410/review...,NaN
28329,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-01-30T00:00:00.000Z,NaN,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,"Very nice for light internet browsing, keeping...",You get a lot for the price!,suzannalicious,http://reviews.bestbuy.com/3545/5620410/review...,NaN
28330,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2017-02-17T00:00:00.000Z,NaN,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",True,NaN,0.0,5,http://reviews.bestbuy.com/3545/5620410/review...,This Tablet does absolutely everything I want!...,You get the entire World for less than $100!,SandyJ,http://reviews.bestbuy.com/3545/5620410/review...,NaN
28331,AVqkIhxunnc1JgDc3kg_,2017-03-06T14:59:43Z,2019-02-23T02:49:38Z,"Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...",B018T075DC,Amazon,"Fire Tablets,Tablets,All Tablets,Amazon Tablet...",Electronics,https://www.upccodesearch.com/images/barcode/0...,"amazon/b018t075dc,firehd8tabletwithalexa8hddis...",Amazon,53-004485,2016-12-23T00:00:00.000Z,NaN,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",True,NaN,0.0,4,http://reviews.bestbuy.com/3545/5620410/review...,"At ninety dollars, the expectionations are low...",You get what your paying for,Hung,http://reviews.bestbuy.com/3545/5620410/review...,NaN


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33332 entries, 0 to 28331
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   33332 non-null  object 
 1   dateAdded            33332 non-null  object 
 2   dateUpdated          33332 non-null  object 
 3   name                 33332 non-null  object 
 4   asins                33332 non-null  object 
 5   brand                33332 non-null  object 
 6   categories           33332 non-null  object 
 7   primaryCategories    33332 non-null  object 
 8   imageURLs            33332 non-null  object 
 9   keys                 33332 non-null  object 
 10  manufacturer         33332 non-null  object 
 11  manufacturerNumber   33332 non-null  object 
 12  reviews.date         33332 non-null  object 
 13  reviews.dateAdded    1052 non-null   object 
 14  reviews.dateSeen     33332 non-null  object 
 15  reviews.doRecommend  21086 non-null 

In [10]:
data.drop('reviews.didPurchase', inplace=True, axis=1)

In [11]:
data.drop('keys', inplace=True, axis=1)

In [12]:
data.drop('reviews.dateAdded', inplace=True, axis=1)

In [13]:
data.drop('categories', inplace=True, axis=1)

In [14]:
data.drop('sourceURLs', inplace=True, axis=1)

In [15]:
data.drop('reviews.sourceURLs', inplace=True, axis=1)

In [16]:
data.drop('reviews.id', inplace=True, axis=1)

In [17]:
data.drop('imageURLs', inplace=True, axis=1)

In [18]:
data.rename(columns={'reviews.doRecommend': 'doRecommend', 'reviews.numHelpful': 'numHelpful'}, inplace=True)

In [19]:
data.isnull().sum()

id                        0
dateAdded                 0
dateUpdated               0
name                      0
asins                     0
brand                     0
primaryCategories         0
manufacturer              0
manufacturerNumber        0
reviews.date              0
reviews.dateSeen          0
doRecommend           12246
numHelpful            12217
reviews.rating            0
reviews.text              0
reviews.title            13
reviews.username          0
dtype: int64

In [20]:
data.duplicated().sum()

95

In [21]:
data.drop_duplicates(inplace=True)

In [22]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
val = imputer.fit_transform(np.array(data['doRecommend']).reshape(-1, 1))

val = pd.DataFrame(val)
data['doRecommend'] = val[0]

In [23]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
val = imputer.fit_transform(np.array(data['numHelpful']).reshape(-1, 1))

val = pd.DataFrame(val)
data['numHelpful'] = val[0]

In [24]:
data.dropna(inplace=True)

In [25]:
data.isnull().sum()

id                    0
dateAdded             0
dateUpdated           0
name                  0
asins                 0
brand                 0
primaryCategories     0
manufacturer          0
manufacturerNumber    0
reviews.date          0
reviews.dateSeen      0
doRecommend           0
numHelpful            0
reviews.rating        0
reviews.text          0
reviews.title         0
reviews.username      0
dtype: int64

In [26]:
data.shape

(33224, 17)

In [27]:
data['id'].value_counts()

AVpgNzjwLJeJML43Kpxn    8343
AVpe7xlELJeJML43ypLz    3728
AVqkIhxunnc1JgDc3kg_    2443
AVqVGWQDv8e3D1O-ldFr    2370
AVph0EeEilAPnD_x9myq    1986
                        ... 
AVpiZQ8r1cnluZ0-M3MN       1
AVzoFq6UvKc47QAVe9NB       1
AV2ElNnuvKc47QAVouhY       1
AVpe5Q3sLJeJML43xt5X       1
AVzvV7FHglJLPUi8GzhD       1
Name: id, Length: 69, dtype: int64

In [28]:
Label_Encoder = LabelEncoder()
labels = Label_Encoder.fit_transform(data.id)
data['id'] = labels

In [29]:
data['dateAdded'].value_counts()

2015-10-30T08:59:32Z    8343
2017-03-03T16:55:53Z    3799
2015-12-03T01:23:41Z    3728
2017-01-11T06:58:33Z    3662
2017-03-06T14:59:43Z    3240
2016-06-08T03:50:06Z    1405
2016-04-10T22:00:34Z    1395
2017-06-24T05:30:23Z    1013
2017-03-06T14:59:25Z     846
2017-01-04T03:51:17Z     826
2018-02-02T02:30:22Z     642
2018-05-02T14:01:51Z     590
2016-03-31T21:08:24Z     587
2017-03-06T14:59:44Z     576
2017-05-20T01:34:04Z     526
2017-03-03T16:56:06Z     427
2017-03-03T16:56:05Z     340
2017-03-06T14:59:52Z     209
2016-08-01T02:44:31Z     195
2018-04-13T02:03:15Z     192
2017-06-22T02:09:02Z     160
2017-03-06T14:59:42Z     112
2017-06-24T05:30:24Z      82
2015-12-09T08:57:06Z      61
2015-12-02T05:30:10Z      57
2017-11-03T23:17:55Z      26
2015-05-11T12:33:03Z      25
2015-01-22T12:25:23Z      21
2017-06-28T15:41:38Z      13
2015-12-01T05:29:38Z      12
2015-11-06T02:16:04Z       9
2015-10-30T10:58:06Z       9
2018-11-24T02:10:04Z       7
2018-06-13T19:37:34Z       7
2018-01-30T04:

In [30]:
data['dateUpdated'].value_counts()

2019-04-25T09:08:16Z    8343
2019-03-09T07:13:43Z    4301
2019-04-24T02:17:42Z    3728
2019-02-23T02:49:38Z    2443
2019-02-25T02:23:15Z    1569
                        ... 
2019-03-09T07:21:48Z       1
2019-03-26T16:55:27Z       1
2019-02-25T02:20:02Z       1
2019-02-25T02:34:23Z       1
2019-04-25T08:39:23Z       1
Name: dateUpdated, Length: 69, dtype: int64

In [31]:
data.isnull().sum()

id                    0
dateAdded             0
dateUpdated           0
name                  0
asins                 0
brand                 0
primaryCategories     0
manufacturer          0
manufacturerNumber    0
reviews.date          0
reviews.dateSeen      0
doRecommend           0
numHelpful            0
reviews.rating        0
reviews.text          0
reviews.title         0
reviews.username      0
dtype: int64

In [32]:
data['name'].value_counts()

AmazonBasics AAA Performance Alkaline Batteries (36 Count)                               8343
AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary           3728
Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers        2443
All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black    2370
Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case                   1986
                                                                                         ... 
Echo Dot (Previous generation)                                                              1
AmazonBasics Single-Door Folding Metal Dog Crate - Large (42x28x30 Inches)                  1
AmazonBasics Silicone Hot Handle Cover/Holder - Red                                         1
Two Door Top Load Pet Kennel Travel Crate Dog Cat Pet Cage Carrier Box Tray 23"             1
AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capac

In [33]:
labels = Label_Encoder.fit_transform(data.name)
data['name'] = labels

In [34]:
data['asins'].value_counts()

B00QWO9P0O,B00LH3DMUO               8343
B00QWO9P0O,B01IB83NZG,B00MNV8E0C    3728
B018T075DC                          2443
B018SZT3BK,B01AHB9CN2               2370
B018Y22C2Y                          1986
                                    ... 
B00VKTZFB4                             1
B00QAVO43C                             1
B010RLCH2U                             1
B00OP6SMCI                             1
B01L7XWEQQ                             1
Name: asins, Length: 70, dtype: int64

In [35]:
labels = Label_Encoder.fit_transform(data.asins)
data['asins'] = labels

In [36]:
data['brand'].value_counts()

Amazon          21045
Amazonbasics    12169
AmazonBasics       10
Name: brand, dtype: int64

In [37]:
data.loc[data['brand'] == 'AmazonBasics']

,id,dateAdded,dateUpdated,name,asins,brand,primaryCategories,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,doRecommend,numHelpful,reviews.rating,reviews.text,reviews.title,reviews.username
13376,31,2015-12-14T23:13:27Z,2019-03-09T07:21:48Z,44,31,AmazonBasics,Home & Garden,AmazonBasics,DHCD002,2017-12-21T00:00:00.000Z,2018-06-01T00:00:00Z,True,0.0,5,I work for Nespresso so I have a lot of capsul...,Nespresso capsule drawer,sheilajw1
19390,4,2015-10-16T18:48:32Z,2019-02-25T02:34:23Z,47,16,AmazonBasics,Animals & Pet Supplies,AmazonBasics,9001-42A,2017-07-03T00:00:00.000Z,"2017-09-08T00:00:00Z,2017-09-15T00:00:00Z,2017...",True,0.0,5,"It's great, study enough to hold the beast.",Nice cage,mdem8573
19395,12,2015-12-02T03:14:48Z,2019-02-25T02:25:00Z,42,15,AmazonBasics,Animals & Pet Supplies,AmazonBasics,9001-36B,2017-09-04T00:00:00.000Z,2017-11-03T00:00:00Z,True,0.0,4,I am very pleased with the dog crate. Very stu...,Better than expected.,ansl-alex
19396,12,2015-12-02T03:14:48Z,2019-02-25T02:25:00Z,42,15,AmazonBasics,Animals & Pet Supplies,AmazonBasics,9001-36B,2017-03-21T00:00:00.000Z,2017-11-03T00:00:00Z,True,0.0,3,The price of the item was very good compared t...,Great price,jeff_dias1
25878,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2013-02-25T00:00:00Z,2017-01-28T00:00:00Z,True,0.0,4,"Like a lot of reviewers here, I struggled to f...",An excellent keyboard -- with one significant ...,Rob Blackwell
25879,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2012-05-22T00:00:00Z,2016-03-21T00:00:00Z,True,0.0,4,Using this with my Samsung Galaxy Note. Overal...,"Good keyboard, but where's the case Here's a s...",LeoB
25880,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2011-12-05T00:00:00Z,2017-01-28T00:00:00Z,True,0.0,5,It's a bit smaller than Apple's wireless keybo...,Great keyboard,debugy2k
25881,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2012-11-28T00:00:00Z,2015-12-03T00:00:00Z,True,0.0,4,I'm typing this review on my Kindle Fire HD 8....,I love this keyboard,Dave Edmiston
25882,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2012-10-30T00:00:00Z,2016-03-21T00:00:00Z,True,0.0,5,Read more,"previous problems with Fire HD 7 now fixed, gr...",Donald M. Shepherd
25883,30,2015-11-29T21:24:13Z,2019-02-24T21:41:04Z,41,2,AmazonBasics,Electronics,AmazonBasics,KT-1081A,2012-11-10T00:00:00Z,2015-12-03T00:00:00Z,True,1.0,4,I waited to get this keyboard until I did a li...,Very nice Bluetooth keyboard for the Kindle Fi...,Brian Hartman


In [38]:
data['brand']=data['brand'].replace({'AmazonBasics':'Amazonbasics'})

In [39]:
labels = Label_Encoder.fit_transform(data.brand)
data['brand'] = labels

In [40]:
data['primaryCategories'].value_counts()

Electronics                    17204
Health & Beauty                12071
Toys & Games,Electronics        1676
Electronics,Hardware            1423
Office Supplies,Electronics      622
Electronics,Media                209
Office Supplies                    9
Animals & Pet Supplies             6
Electronics,Furniture              2
Home & Garden                      2
Name: primaryCategories, dtype: int64

In [41]:
labels = Label_Encoder.fit_transform(data.primaryCategories)
data['primaryCategories'] = labels

In [42]:
data['manufacturer'].value_counts()

Amazon                     21022
AmazonBasics               12179
Amazon Digital Services       18
Amazon.com                     5
Name: manufacturer, dtype: int64

In [43]:
labels = Label_Encoder.fit_transform(data.manufacturer)
data['manufacturer'] = labels

In [44]:
data['manufacturerNumber'].value_counts()

HL-002619           8343
LR6G0748FFPAB-US    3728
53-004484           3167
53-004485           2443
53-004755           1986
                    ... 
9001-42A               1
DHCD002                1
AOKE-16082501          1
6009-S                 1
53-003736              1
Name: manufacturerNumber, Length: 74, dtype: int64

In [45]:
labels = Label_Encoder.fit_transform(data.manufacturerNumber)
data['manufacturerNumber'] = labels

In [46]:
data['doRecommend'].value_counts()

True     32275
False      949
Name: doRecommend, dtype: int64

In [47]:
labels = Label_Encoder.fit_transform(data.doRecommend)
data['doRecommend'] = labels

In [48]:
data.rename(columns={'reviews.date': 'reviews_date', 'reviews.dateSeen': 'reviews_dateSeen'}, inplace=True)

In [49]:
data['reviews_date'].value_counts()

2017-01-23T00:00:00.000Z    656
2017-01-06T00:00:00.000Z    630
2017-01-18T00:00:00.000Z    543
2017-01-16T00:00:00.000Z    535
2016-12-23T00:00:00.000Z    449
                           ... 
2014-08-11T00:00:00.000Z      1
2017-01-01T00:04:32.000Z      1
2014-11-21T00:00:00.000Z      1
2015-09-10T00:00:00Z          1
2017-09-01T15:08:55.000Z      1
Name: reviews_date, Length: 1344, dtype: int64

In [50]:
data['reviews_dateSeen'].value_counts()

2017-08-28T00:00:00Z                                                                                                                                                                            8343
2017-06-28T00:00:00Z                                                                                                                                                                            3728
2017-04-30T00:00:00Z,2017-06-07T00:00:00Z                                                                                                                                                       2947
2017-04-30T00:00:00Z,2017-06-07T00:00:00Z,2017-04-26T00:00:00Z,2017-08-06T00:00:00Z,2017-05-10T00:00:00Z,2017-08-09T00:00:00Z                                                                   1650
2017-04-26T00:00:00Z,2017-06-04T00:00:00Z,2017-06-03T00:00:00Z                                                                                                                                  1492
               

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33224 entries, 0 to 28331
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  33224 non-null  int64  
 1   dateAdded           33224 non-null  object 
 2   dateUpdated         33224 non-null  object 
 3   name                33224 non-null  int64  
 4   asins               33224 non-null  int64  
 5   brand               33224 non-null  int64  
 6   primaryCategories   33224 non-null  int64  
 7   manufacturer        33224 non-null  int64  
 8   manufacturerNumber  33224 non-null  int64  
 9   reviews_date        33224 non-null  object 
 10  reviews_dateSeen    33224 non-null  object 
 11  doRecommend         33224 non-null  int64  
 12  numHelpful          33224 non-null  float64
 13  reviews.rating      33224 non-null  int64  
 14  reviews.text        33224 non-null  object 
 15  reviews.title       33224 non-null  object 
 16  revi

In [52]:
data.sample(5)

,id,dateAdded,dateUpdated,name,asins,brand,primaryCategories,manufacturer,manufacturerNumber,reviews_date,reviews_dateSeen,doRecommend,numHelpful,reviews.rating,reviews.text,reviews.title,reviews.username
5776,22,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,39,18,1,5,3,64,2017-03-12T00:00:00.000Z,2017-08-28T00:00:00Z,1,0.0,5,Long lasting great product,GREAT!,ByMaynor A.
3515,45,2017-03-06T14:59:43Z,2018-02-13T21:53:06Z,10,45,0,1,0,15,2017-01-21T00:00:00.000Z,"2017-04-30T00:45:00.000Z,2017-06-07T09:04:00.000Z",1,0.0,4,It's alright to browse the Internet and watch ...,It's alright,JMH28
22529,33,2016-06-08T03:50:06Z,2019-02-25T02:05:58Z,65,38,0,1,0,21,2017-01-16T00:00:00.000Z,"2017-08-27T00:00:00Z,2017-08-09T00:00:00Z,2017...",1,0.0,4,We are used to iPads in our house. This has ta...,Nice product for the price,Momofthree
21273,41,2017-03-06T14:59:25Z,2019-02-25T02:20:29Z,66,37,0,1,0,22,2016-10-07T00:00:00.000Z,"2017-04-30T00:00:00Z,2017-06-07T00:00:00Z",1,0.0,5,Good inexpensive device. Still in touch with m...,Great Gift,WifiG
1686,22,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,39,18,1,5,3,64,2017-01-17T00:00:00.000Z,2017-08-28T00:00:00Z,1,0.0,5,Fast delivery. Will order again,Five Stars,ByAngie


In [53]:
lower_bound = 0
upper_bound = 5
x = data[(data['reviews.rating'] < lower_bound) | (data['reviews.rating'] > upper_bound)].index
print(len(x))

0
